In [1]:
from env import env
import pandas as pd
import random as rd


In [2]:
df = pd.read_csv('results/bike-results/result.csv')

### Get od pair

In [6]:
# try later: https://geobgu.xyz/py/shapely.html

lat = df['station_lat'].to_list()
lng = df['station_lng'].to_list()

rd.seed(31)
x = rd.sample(range(0,2191),2)

originCoord = (lat[x[0]],lng[x[0]])
origin = f'{originCoord[0]},{originCoord[1]}'

destinationCoord = (lat[x[1]],lng[x[1]])
destination = f'{destinationCoord[0]},{destinationCoord[1]}'

print(origin)
print(destination)

41.9106,-87.6494
41.76,-87.58


### Here requests

Car co2 and fuel consumption link: https://developer.here.com/documentation/routing-api/dev_guide/topics/use-cases/calculate-fuel-consumption.html

In [3]:
from haversine import haversine, Unit
import flexpolyline as fp
import urllib.request
import json, sys

In [5]:
# to be determined
#colors = {
#    'pedestrian':'blue',
#    'bus':'yellow',
#    'bicycle':'green'
#}

# add colors to different sections of route depending on transport mode

def coord_reader(response):
    latList = []
    lngList = []

    for i in response['routes'][0]['sections']:
        try:
            polyline = fp.decode(i['polyline'])

            for coord in polyline:
                latList.append(coord[0])
                lngList.append(coord[1])
        
        except:
            pass

        latList.append(i['arrival']['place']['location']['lat'])
        lngList.append(i['arrival']['place']['location']['lng'])
    
    return latList,lngList

latList,lngList = coord_reader(hereResponse)

#Route midpoint
middle = delta = hereResponse['routes'][0]['sections'][0]['summary']['length']/2
zipList = list(zip(latList,lngList))
before = after = None
dist = 0

for i in range(len(zipList)-1):
    dist += haversine(zipList[i],zipList[i+1], unit=Unit.METERS)
    sub = abs(middle-dist)
    if sub < delta:
        delta = sub
        before = zipList[i+1]
    if sub > delta:
        after = zipList[i+1]
        break

midpointCoord = ((before[0]+after[0])/2,(before[1]+after[1])/2)
midpoint = f'{midpointCoord[0]},{midpointCoord[1]}'

print(f'route midpoint for multimodal: {midpointCoord}')

In [8]:
# get closest bike share point on coordinate
def closest(coord: list): #### need refactor!!
    data = pd.read_csv('results/bike-results/result.csv')
    dist = sys.float_info.max
    waypoint = ()
    for ind,row in data.iterrows():
        distTest = haversine(coord,(row['station_lat'],row['station_lng']))
        if distTest < dist:
            dist = distTest
            waypoint = (row['station_lat'],row['station_lng'])
    return waypoint

In [42]:
def car_routing(strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'
    hereKey = env('HERE_API_KEY')
    
    fuelConsumption = '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

    url = 'https://router.hereapi.com/v8/routes?transportMode=%s%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % ('car',fuelConsumption,query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return json.loads(response)

In [43]:
response = car_routing((origin,destination))
print(response)

{'routes': [{'id': '358a01bc-f967-47ce-aef4-f24122415d70', 'sections': [{'id': '4ccbaddb-7549-4f3c-851d-c9baa100e349', 'type': 'vehicle', 'departure': {'time': '2023-09-25T09:58:44-05:00', 'place': {'type': 'place', 'location': {'lat': 41.91057, 'lng': -87.64949, 'elv': 149.0}, 'originalLocation': {'lat': 41.9106, 'lng': -87.6494}}}, 'arrival': {'time': '2023-09-25T10:27:20-05:00', 'place': {'type': 'place', 'location': {'lat': 41.7600018, 'lng': -87.5798541, 'elv': 147.0}, 'originalLocation': {'lat': 41.76, 'lng': -87.5800001}}}, 'summary': {'duration': 1716, 'length': 21430, 'consumption': 5.1527, 'baseDuration': 1384, 'co2Emission': 13.897}, 'polyline': 'BG0yg-vCjt2lnFrJUjDArJUvWAAkuCwHA0ZnBkhBnBU4cAwMAsOUoQAgeAoGU8LUofU4wBAkrBAsOoBssBoB8a8B0ZvCsEAsJUsJUsYA8LAsJAwRUkNAoGA4cAwCAkmBoB4mBoBwlBUoaAgKU8VA4IAgUUoQA0PT4IUkXA4cA8fA0ZAsEU0ZUsJAsEAoLwCoGAwHAwMUkS0eA0oBTokBnB0FsJsJgPwCoQ4D0U8BsJUoGoB0PTsTToLnBwHnBoGnB0F7B8GvC4I3DoLjDkIzFkNjIwRjI8QrJ0UjDwH3D4IvCoGjD4IvCkI7BkInBkIT0FTwHTwMAwMUgK

In [10]:
def transit_routing(strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'
    hereKey = env('HERE_API_KEY')

    # change fuel consumption for transport if possible
    #fuelConsumption = '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

    # multimodal here url
    # transposrt types:
    # vehicle
    # transit
    # taxi
    # rented
    
    url = 'https://intermodal.router.hereapi.com/v8/routes?%s&transit[enable]=entireRoute&rented[enable]=&taxi[enable]=&vehicle[enable]=&alternatives=%d&apiKey=%s' % (query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return json.loads(response)


In [11]:
response = transit_routing((origin,destination))
print(response)

{'routes': [{'id': 'R0', 'sections': [{'id': 'R0-S0', 'type': 'pedestrian', 'departure': {'time': '2023-09-26T08:01:00-05:00', 'place': {'type': 'place', 'location': {'lat': 41.9106, 'lng': -87.6494}}}, 'arrival': {'time': '2023-09-26T08:02:00-05:00', 'place': {'name': 'North/Clybourn', 'type': 'accessPoint', 'location': {'lat': 41.91073, 'lng': -87.64925}}}, 'transport': {'mode': 'pedestrian'}}, {'id': 'R0-S1', 'type': 'pedestrian', 'departure': {'time': '2023-09-26T08:02:00-05:00', 'place': {'name': 'North/Clybourn', 'type': 'accessPoint', 'location': {'lat': 41.91073, 'lng': -87.64925}}}, 'arrival': {'time': '2023-09-26T08:03:00-05:00', 'place': {'name': 'North/Clybourn', 'type': 'station', 'location': {'lat': 41.910655, 'lng': -87.649177}, 'id': '418811321'}}, 'spans': [{'walkAttributes': ['indoor']}], 'transport': {'mode': 'pedestrian'}}, {'id': 'R0-S2', 'type': 'transit', 'departure': {'time': '2023-09-26T08:03:00-05:00', 'place': {'name': 'North/Clybourn', 'type': 'station', 'lo

In [123]:
# for pedestrian: https://developer.here.com/documentation/routing-api/dev_guide/topics/pedestrian-defined.html

def bicycle_routing(strCoord,floatCoord,data,maxAlternatives=1):
    hereKey = env('HERE_API_KEY')

    newOrigin = closest(floatCoord[0]) # data
    newDestination = closest(floatCoord[1]) # data

    mode = ['pedestrian','bicycle','pedestrian']

    query = [
        f'destination={newOrigin[0]},{newOrigin[1]}&origin={strCoord[0]}',
        f'destination={newDestination[0]},{newDestination[1]}&origin={newOrigin[0]},{newOrigin[1]}',
        f'destination={strCoord[1]}&origin={newDestination[0]},{newDestination[1]}'
    ]

    responses = []

    for i in range(len(mode)):
        if mode[i] == 'pedestrian': altNum = 0
        else: altNum = maxAlternatives
        url = 'https://router.hereapi.com/v8/routes?transportMode=%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % (mode[i],query[i],maxAlternatives,hereKey)
        print(url)
        response = urllib.request.urlopen(url).read().decode('utf-8')
        responses.append(json.loads(response))

    return responses

In [124]:
response = bicycle_routing((origin,destination),(originCoord,destinationCoord),df)

https://router.hereapi.com/v8/routes?transportMode=pedestrian&destination=41.9106,-87.6494&origin=41.9106,-87.6494&alternatives=1&return=summary,polyline&apikey=RhbiMBCRVNDFplxIdn-yII6eZU9upCqvuChA2n7aSXo
https://router.hereapi.com/v8/routes?transportMode=bicycle&destination=41.76,-87.58&origin=41.9106,-87.6494&alternatives=1&return=summary,polyline&apikey=RhbiMBCRVNDFplxIdn-yII6eZU9upCqvuChA2n7aSXo
https://router.hereapi.com/v8/routes?transportMode=pedestrian&destination=41.76,-87.58&origin=41.76,-87.58&alternatives=1&return=summary,polyline&apikey=RhbiMBCRVNDFplxIdn-yII6eZU9upCqvuChA2n7aSXo


In [133]:
print(response[1]['routes'][1]['sections'][2]['polyline'])

BG-xx2vCtkqhnFVCvHoB7fsEze0K36BwM_7BgUnkBsOrToGvHkDjrB8V7agP7a4S3mB4c_JoGrOoL_EoQvHAjIwCnL8GjIgKnBoG3NvHAzUAjI3SUnQjDnQzFrO7GvMjIjInGjIzFrJ7GvR7L3I_EvHjD3SzFzF7B_ET3DoBnLsEjSUzKkDvR0Fna8LvMoGrJgFjSoGzPwC_JnB7LnBjNvCnGvCnG3DvCvCjDvC_JzKnG7G3NzPrJzK7B7BnL_JnGrE3D7B3D7B7G7BvHTrJTvMU_TA3NUzyBoBjSU_OUvtC8B7VUzPAnxC8B7QU3SU3rBoBzKUrJA_YU_EAvHAjXA3DAjNAnGA_EA7foBjNUvCArJUrEAjNU_OAjXU7BAjcUvbU7LArOU7BA_OU3NUnaUrYUzFAT7pBTzFArTUjNTvMA_JA7LA7GArTAvMTvbAzKT_nBjpBgB


In [150]:
routes = []

for i in response[1]['routes']:
    aux = []
    for j in i['sections']:
        aux.append((j['transport']['mode'],fp.decode(j['polyline'])))
    routes.append(aux)
for i in routes:
    print(i)

('bicycle', [(41.91057, -87.64949), (41.91042, -87.64948), (41.91037, -87.64948), (41.91022, -87.64947), (41.91024, -87.6493), (41.9105, -87.64894), (41.91039, -87.64879), (41.91023, -87.64856), (41.90998, -87.64822), (41.90969, -87.64783), (41.9095, -87.64757), (41.90924, -87.64723), (41.909, -87.64689), (41.90894, -87.64681), (41.90878, -87.64657), (41.90871, -87.64649), (41.90866, -87.64641), (41.90861, -87.64634), (41.90851, -87.6462), (41.90846, -87.64613), (41.90805, -87.64553), (41.90787, -87.64528), (41.90773, -87.64509), (41.90756, -87.64484), (41.90742, -87.64465), (41.90737, -87.64458), (41.90719, -87.64432), (41.9071, -87.6442), (41.90703, -87.6441), (41.90691, -87.64393), (41.90685, -87.64385), (41.90674, -87.64369), (41.90645, -87.64331), (41.90621, -87.64296), (41.90577, -87.64233), (41.90561, -87.6421), (41.90557, -87.64204), (41.90522, -87.64155), (41.90507, -87.64135), (41.90504, -87.64131), (41.90489, -87.64109), (41.9046, -87.64068), (41.90437, -87.64034), (41.90369

In [151]:
#def multimodal_routing(strCoord,floatCoord,data,maxAlternatives=0):
print(len(routes))

4


### Route map print

In [ ]:
from bs4 import BeautifulSoup as bs
import gmplot

In [11]:
colors = {
    'pedestrian':'blue',
    'bicycle':'green'
}

def map_print(fileName,originCoord,routes,midpointCoord,pointMarker=False):

    outputMap = gmplot.GoogleMapPlotter(originCoord[0],originCoord[1], 12)

    #for i in range(len(routes)):

    outputMap.plot(latList, lngList, edge_width=3)
    mapUrl = f'results/map_plot/{fileName}.html'

    if pointMarker:
        outputMap.marker(midpointCoord[0],midpointCoord[1],color='green')

    outputMap.draw(mapUrl)

    # adding the api key into html file
    page = open(mapUrl)
    soup = bs(page.read(),"html.parser")
    srcKey  = f"https://maps.googleapis.com/maps/api/js?key={env('GOOGLE_MAPS_KEY')}&libraries=visualization&sensor=true_or_false"
    line = soup.find('script',type="text/javascript")
    line['src'] = srcKey

    with open(mapUrl, "wb") as f_output:
        f_output.write(soup.prettify("utf-8"))  

In [14]:
map_print('Here_map',originCoord,latList,lngList,midpointCoord)

In [ ]:
# tomtom test with car
#query = f'{origin}:{destination}'
#tomtomkey = env('TOM_TOM_KEY')

#url='https://api.tomtom.com/routing/1/calculateRoute/%s/json?&key=%s&sectionType=traffic&instructionsType=coded&maxAlternatives=%d&traffic=true&travelMode=car' % (query, tomtomkey, maxAlternatives)